In [ ]:
!pip install face_alignment ffmpeg-python pyyaml==5.1.2 pydub flask_ngrok
!git clone https://github.com/hemanth-nag/AV-Speech-Separation-API
%cd AV-Speech-Separation-API

from video_features.utils import get_position, transformation_from_points
from video_features.model import LipNet

from AVDPRNN.data_loader import audio_reader
import os
import torch
from torch.nn.parallel import data_parallel
from AVDPRNN.model.avdprnn_model import Dual_RNN_model
from AVDPRNN.config.option import parse

import sys
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math
import os
import numpy as np
import time
from video_features.model import LipNet
import torch.optim as optim
import re
import json
import tempfile
import shutil
import cv2
import face_alignment
import matplotlib.pyplot as plt
import uuid
from tqdm import tqdm, trange
from pathlib import Path
from joblib import Parallel, delayed
from time import perf_counter
import subprocess
import ffmpeg
import matplotlib.pyplot as plt
%matplotlib inline

class inference():
  def __init__(self):
    self.model = LipNet()
    self.model = self.model.cuda()
    self.model.eval()
    self.net = nn.DataParallel(self.model).cuda()
    weights = 'video_features/pretrain/LipNet_unseen_loss_0.44562849402427673_wer_0.1332580699113564_cer_0.06796452465503355.pt'
    pretrained_dict = torch.load(weights)
    model_dict = self.model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict.keys() and v.size() == model_dict[k].size()}
    missed_params = [k for k, v in model_dict.items() if not k in pretrained_dict.keys()]
    model_dict.update(pretrained_dict)
    self.model.load_state_dict(model_dict)

  def avss(self, file_path, coordinates):
    print("Converting Video to frames")
    p = tempfile.mkdtemp()
    stream = ffmpeg.input(file_path, nostdin=None)
    stream = ffmpeg.output(stream, f'{p}/%d.jpg', **{'qscale:v': 2, 'r':25}, loglevel="quiet")

    try:
        out, err = ffmpeg.run(stream)
    except:
        print("FFMPEG CRASHED")
        sys.exit(1)

    if err:
        print(err)
        sys.exit(1)

    files = os.listdir(p)
    files = sorted(files, key=lambda x: int(os.path.splitext(x)[0]))
    #plt.imshow(cv2.cvtColor(cv2.imread(os.path.join(p, '1.jpg')), cv2.COLOR_BGR2RGB))

    print("Face a")
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device=f'cuda:{0}')
    points = fa.get_landmarks_from_directory(p, show_progress_bar=True)
    points = sorted(points.items(), key=lambda x: int(Path(x[0]).stem))

    videos = [[], []]

    front256 = get_position(256)
    for filename, point_faces in tqdm(points):
        first_was_here, second_was_here = False, False
        for point in point_faces:        
            if first_was_here and second_was_here:
                print("Too many faces")
                break
            if coordinates[0][0] < max(point[:, 0]) < coordinates[0][1]:
                idx = 0
                first_was_here = True
            elif coordinates[1][0]< max(point[:, 0]) < coordinates[1][1]:
                idx = 1
                second_was_here = True
            else:
                continue
            if(point is not None):
                shape = np.array(point)
                shape = shape[17:]
                M = transformation_from_points(np.matrix(shape), np.matrix(front256))

                img = cv2.warpAffine(cv2.imread(filename), M[:2], (256, 256))
                (x, y) = front256[-20:].mean(0).astype(np.int32)
                w = 160//2
                img = img[y-w//2:y+w//2,x-w:x+w,...]
                img = cv2.resize(img, (128, 64))
                videos[idx].append(img)
            elif videos:
                videos[idx].append(videos[idx][-1])
            else:
                print("First frame is missing", file)
                
        if not first_was_here:
            print('first none', filename)
        if not second_was_here:
            print('second none', filename)
        
    if not videos[idx]:
        print("No faces in video")
        sys.exit(1)
            
    for idx in range(2):
        videos[idx] = np.stack(videos[idx], axis=0).astype(np.float32)
        videos[idx] = torch.FloatTensor(videos[idx].transpose(3, 0, 1, 2)) / 255.0

    face0_features = self.model(videos[0][None,...].cuda(), return_video_features=True).detach()
    face1_features = self.model(videos[1][None,...].cuda(), return_video_features=True).detach()

    reader = audio_reader.get_default_audio_reader()
    mix_audio = reader.load(file_path)
    mix_audio = reader.to_tensor(mix_audio)
    opt = parse('AVDPRNN/config/Dual_RNN/train.yaml')
    opt['Dual_Path_RNN']['upsample_size'] = mix_audio.shape[1] - 1
    net = Dual_RNN_model(**opt['Dual_Path_RNN'])

    dicts = torch.load('AVDPRNN/saved_model/trained_weights.pt', map_location='cpu')
    net.load_state_dict(dicts["model_state_dict"])
    net = net.to('cuda')

    with torch.no_grad():
        batch = {"mix_audios": mix_audio.cuda(),
                "first_videos_features": face0_features.transpose(1, 0).cuda(),
                "second_videos_features": face1_features.transpose(1, 0).cuda(),
                "mix_noised_audios": mix_audio.cuda()}
        res = net(batch)
    import soundfile as sf
    for idx, s in enumerate(res):
        os.makedirs('output_audio/', exist_ok=True)
        filename="output_audio/"+f'test_{idx}.wav'
        sf.write(filename, s.detach().cpu().numpy().T, opt['datasets']['audio_setting']['sample_rate'])


Cloning into 'AV-Speech-Separation-API'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 41 (delta 7), reused 26 (delta 2), pack-reused 0
Unpacking objects: 100% (41/41), done.
/root/AV-Speech-Separation-API


In [ ]:
reader = audio_reader.get_default_audio_reader()
mix_audio = reader.load(file)
mix_audio = reader.to_tensor(mix_audio)
opt = parse('AVDPRNN/config/Dual_RNN/train.yaml')
opt['Dual_Path_RNN']['upsample_size'] = mix_audio.shape[1] - 1
net = Dual_RNN_model(**opt['Dual_Path_RNN'])

dicts = torch.load('AVDPRNN/saved_model/trained_weights.pt', map_location='cpu')
net.load_state_dict(dicts["model_state_dict"])
net = net.to('cuda')

with torch.no_grad():
    batch = {"mix_audios": mix_audio.cuda(),
             "first_videos_features": face0_features.transpose(1, 0).cuda(),
             "second_videos_features": face1_features.transpose(1, 0).cuda(),
             "mix_noised_audios": mix_audio.cuda()}
    res = net(batch)
import soundfile as sf
for idx, s in enumerate(res):
    os.makedirs('output_audio/', exist_ok=True)
    filename="output_audio/"+f'test_{idx}.wav'
    sf.write(filename, s.detach().cpu().numpy().T, opt['datasets']['audio_setting']['sample_rate'])

RuntimeError: ignored

In [ ]:
avs = inference()

loaded params/tot params:24/24
miss matched params:[]


Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


  0%|          | 0.00/85.7M [00:00<?, ?B/s]

Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip


  0%|          | 0.00/91.9M [00:00<?, ?B/s]

  0%|          | 0/1494 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
 11%|█         | 162/1494 [00:08<01:14, 17.84it/s]

...

Too many faces


 36%|███▌      | 531/1494 [00:29<00:54, 17.77it/s]

Too many faces
Too many faces


100%|██████████| 1494/1494 [01:21<00:00, 18.32it/s]


RuntimeError: ignored

In [ ]:

from flask_ngrok import run_with_ngrok
import os
import urllib.request
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
import time

UPLOAD_FOLDER = './'

avs = inference()

app = Flask(__name__)
run_with_ngrok(app)
#app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
#app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
	
@app.route('/', methods=['POST'])
def upload():
	if 'file' not in request.files:
		return 'No File'
	else:
		file = request.files['file']
		filename = 'vid.mp4'
		file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
	  avs.avss(file_path='/content/vid.mp4', coordinates=[[0,900], [901,1900]])
    return 'Success'
if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7a4b-34-121-202-47.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Sep/2021 16:23:47] "POST / HTTP/1.1" 200 -


In [ ]:
file = '/content/second_speaker.mp4'
p = tempfile.mkdtemp()

stream = ffmpeg.input(file, nostdin=None)
stream = ffmpeg.output(stream, f'{p}/%d.jpg', **{'qscale:v': 2, 'r':25}, loglevel="quiet")

try:
    out, err = ffmpeg.run(stream)
except Exception as e:
    print("FFMPEG CRASHED",e)
    sys.exit(1)

if err:
    print(err)
    sys.exit(1)

In [ ]:
ls

AVDPRNN/  README.md  video_features/
